In [7]:
# Create input dataframe for future predictions

import pandas as pd
from itertools import product

# Dates
dates = ['203001','203002','203003','203004','203005','203006','203007','203008','203009','203010','203011','203012',]

# Range: -.93 (~195102) --> .5 --> 2.52 (201602)
global_land_temps = [.5, 1.1, 1.7, 2.3, 2.9]
# Range: -.22 (~195412) --> .3 --> 1.04 (202402)
global_ocean_temps = [0.3, 0.7, 0.9, 1.1, 1.3]
# Range Deseasonalized: 314.44 (195803) --> 359 (1996) --> 426.12 (202412)
co2_levels = [350, 375, 400, 425, 450]

# All 50
state_codes = [
    'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

# Generate all unique combinations using Cartesian product
combinations = list(product(dates, global_land_temps, global_ocean_temps, co2_levels, state_codes))

# Create DataFrame
df = pd.DataFrame(combinations, columns=[
    'date', 'global_land_temp', 'global_ocean_temp', 'co2_levels', 'state_code'
])

#DataFrame
print(df)


# 124416 rows


         date  global_land_temp  global_ocean_temp  co2_levels state_code
0      203001               0.5                0.3         350         AL
1      203001               0.5                0.3         350         AZ
2      203001               0.5                0.3         350         AR
3      203001               0.5                0.3         350         CA
4      203001               0.5                0.3         350         CO
...       ...               ...                ...         ...        ...
71995  203012               2.9                1.3         450         VA
71996  203012               2.9                1.3         450         WA
71997  203012               2.9                1.3         450         WV
71998  203012               2.9                1.3         450         WI
71999  203012               2.9                1.3         450         WY

[72000 rows x 5 columns]


In [8]:
# Process climate change data into one table containing both global temps and co2 levels 

import pandas as pd
from sqlalchemy import create_engine, text

# Configuration
DB_CONFIG = {
    "username": "agowen",  # Replace with PostgreSQL username
    "password": "Capstone",  # Replace with PostgreSQL password
    "host": "localhost",          # Replace with PostgreSQL host
    "port": 5432,                 # Replace with PostgreSQL port (default: 5432)
    "database": "capstone"      # Replace with PostgreSQL database name                              # Update DB Name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def create_full_climate_change_data_table():
    try:
            
        with engine.connect() as connection:
            
            query = text(f"""
            select * from state_region_mapping
                                 """)
            df_state_info = pd.read_sql(query, connection)
    
        result_df = pd.merge(df,df_state_info, on='state_code',how='inner')


        # Create a new table to store the transformed data
        new_table_name = "_input_predicting_table"                                                        # update new table name if needed 
        result_df.to_sql(new_table_name, engine, if_exists="replace", index=False)
        print(f"Table '{new_table_name}' has been created successfully.")

    except Exception as e:
        print(f"Error: {e}")

def main():
    create_full_climate_change_data_table()

if __name__ == "__main__":
    main()


Table '_input_predicting_table' has been created successfully.
